# Import python libraries

In [1]:
# imports
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline

from sklearn.metrics import confusion_matrix, plot_confusion_matrix

from sklearn.model_selection import GridSearchCV , train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder

import numpy as np

from sklearn.preprocessing import StandardScaler , Normalizer
from sklearn.svm import LinearSVC , SVC
# Import CountVectorizer and TFIDFVectorizer from feature_extraction.text.
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


### Import Train and Test Data

In [2]:
X_train= pd.read_csv("./datasets/X_train.csv" , squeeze = True)
y_train= pd.read_csv("./datasets/y_train.csv", squeeze = True)

X_test= pd.read_csv("./datasets/X_test.csv" , squeeze = True)
y_test= pd.read_csv("./datasets/y_test.csv", squeeze = True)

### Model 6: SVM : Pipeline - GridSearch

In [3]:
# Let's set a pipeline up with two stages:
# 1. CountVectorizer (transformer)
# 2.  SVC (estimator)
pipe = Pipeline([
    ('cvec', CountVectorizer()),
    ('nm', Normalizer()),
    ('sc',StandardScaler(with_mean=False)),
    ('svc', SVC())
])

In [4]:
# Search over the following values of hyperparameters:
# Maximum number of features fit: 2000, 3000, 4000, 5000
# Minimum number of documents needed to include token: 2, 3
# Maximum number of documents needed to include token: 90%, 95%
# Check (individual tokens) and also check (individual tokens and 2-grams).
pipe_params = {
    'cvec__max_features' : [200, 300, 400, 500],
    'cvec__stop_words' : [None,['Sherlock', 'Poirot'], 'english'],
    'cvec__ngram_range' : [(1,1), (1,2)],
    'svc__kernel':( 'linear','rbf'),
    'svc__gamma' :('scale', 'auto' ),
    'svc__C':(1, 10)
}

In [5]:
# Instantiate GridSearchCV.
gs_svc = GridSearchCV(pipe,
                      pipe_params,
                      cv = 5,
                      verbose=1,
                      n_jobs=8)

In [6]:
# Fit GridSearch to training data.
gs_svc.fit(X_train, y_train)

Fitting 5 folds for each of 192 candidates, totalling 960 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    1.5s
[Parallel(n_jobs=8)]: Done 560 tasks      | elapsed:    7.9s
[Parallel(n_jobs=8)]: Done 945 out of 960 | elapsed:   13.2s remaining:    0.2s
[Parallel(n_jobs=8)]: Done 960 out of 960 | elapsed:   13.3s finished


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('cvec', CountVectorizer()),
                                       ('nm', Normalizer()),
                                       ('sc', StandardScaler(with_mean=False)),
                                       ('svc', SVC())]),
             n_jobs=8,
             param_grid={'cvec__max_features': [200, 300, 400, 500],
                         'cvec__ngram_range': [(1, 1), (1, 2)],
                         'cvec__stop_words': [None, ['Sherlock', 'Poirot'],
                                              'english'],
                         'svc__C': (1, 10), 'svc__gamma': ('scale', 'auto'),
                         'svc__kernel': ('linear', 'rbf')},
             verbose=1)

In [7]:
gs_svc_df = pd.DataFrame(gs_svc.cv_results_)

In [8]:
gs_svc.best_estimator_

Pipeline(steps=[('cvec',
                 CountVectorizer(max_features=500, stop_words='english')),
                ('nm', Normalizer()), ('sc', StandardScaler(with_mean=False)),
                ('svc', SVC(C=1, kernel='linear'))])

In [9]:
gs_svc.best_params_

{'cvec__max_features': 500,
 'cvec__ngram_range': (1, 1),
 'cvec__stop_words': 'english',
 'svc__C': 1,
 'svc__gamma': 'scale',
 'svc__kernel': 'linear'}

### Run the model on Test data

In [10]:
# Evaluate the best fit model on the test data.
best_svc = gs_svc.best_estimator_
print(f"Training Score from best SVM: {gs_svc.score(X_train , y_train)}")
print(f"Test Score from best SVM: {gs_svc.score(X_test, y_test)}") 

Training Score from best SVM: 0.9939024390243902
Test Score from best SVM: 0.851145038167939
